<a href="https://colab.research.google.com/github/thunyalak230/GE234_Thunyalak/blob/main/code_repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# @title สร้างกราฟพื้นผิวภูเขา 3D (Surface Plot) พร้อมป้ายบอกความสูง
import numpy as np
import pandas as pd
import plotly.graph_objects as go #วาดกราฟ 3 มิติแบบ Interactive คำว่า go คือการจัดการข้อมูล Matrix ให้มีความยืดหยุ่น
from scipy.interpolate import griddata #scipy.interpolation คือกล่องเครื่องมือและเครื่องมือที่ใช้คือ griddata เพื่อที่จะไม่ให้เกิดช่องว่างในพื้นที่ 3D

# 1. ข้อมูลภูเขา
data = [
    {"name": "ดอยอินทนนท์", "lat": 18.5884, "lon": 98.4873, "height": 2565},
    {"name": "ดอยผ้าห่มปก", "lat": 20.0436, "lon": 99.1466, "height": 2285},
    {"name": "ดอยเชียงดาว", "lat": 19.3956, "lon": 98.8850, "height": 2175},
    {"name": "ภูสอยดาว", "lat": 17.6961, "lon": 100.9472, "height": 2102},
    {"name": "เขาหลวง (ใต้)", "lat": 8.5085, "lon": 99.7314, "height": 1835},
    {"name": "เขาสอยดาวใต้", "lat": 12.9239, "lon": 102.1950, "height": 1675},
    {"name": "ภูชี้ฟ้า", "lat": 19.8496, "lon": 100.4492, "height": 1628},
    {"name": "เขาใหญ่", "lat": 14.4390, "lon": 101.3725, "height": 1351},
    {"name": "ภูกระดึง", "lat": 16.8845, "lon": 101.7925, "height": 1316},
    # เพิ่มจุดสมมติที่ระดับน้ำทะเลรอบๆ เพื่อให้กราฟลาดลงพื้น (Grounding) หากไม่มีอาจเกิดพื้นที่แหว่ง
    #โดยค่าจะกำหนดจากจุดสูงสุดและต่ำสุด
    {"name": "", "lat": 21.0, "lon": 97.0, "height": 0},
    {"name": "", "lat": 21.0, "lon": 106.0, "height": 0},
    {"name": "", "lat": 5.0, "lon": 97.0, "height": 0},
    {"name": "", "lat": 5.0, "lon": 106.0, "height": 0}
]

df = pd.DataFrame(data)

# เตรียมข้อมูลสำหรับสร้างพื้นผิว (Meshgrid)
# สร้างโครงข่าย 100*100 (Grid) เพื่อวาดพื้นผิว
grid_x, grid_y = np.mgrid[min(df.lon):max(df.lon):100j, min(df.lat):max(df.lat):100j]

# ใช้ Cubic Interpolation คือเทคนิคการคำนวณเพื่อสร้าง "เส้นโค้ง" เชื่อมระหว่างจุดข้อมูลครับ
    (df.lon, df.lat),
    df.height,
    (grid_x, grid_y),
    method='cubic',
    fill_value=0 #เปลี่ยนให้พื้นที่ที่ไม่ทราบค่าเป็น 0
)
grid_z[grid_z < 0] = 0 #เปลี่ยนค่าที่มีค่าน้อยกว่า 0 ให้เป็น 0

# 3. สร้างกราฟ 3D
fig = go.Figure()

# Layer 1: พื้นผิวภูเขา (Surface)
fig.add_trace(go.Surface(
    z=grid_z,
    x=grid_x,
    y=grid_y,
    colorscale='Earth', # สีโทนโลก/ภูเขา
    opacity=0.9,
    name='Terrain',
    showscale=False # ซ่อนแถบสีด้านข้างเพื่อความสะอาดตา
))

# กรองเฉพาะจุดที่มีชื่อภูเขา (ตัดจุดสมมติทิ้ง)
real_mountains = df[df['name'] != ""]

# Layer 2: จุดยอดเขาและป้ายชื่อ (Scatter3D)
fig.add_trace(go.Scatter3d(
    x=real_mountains['lon'],
    y=real_mountains['lat'],
    z=real_mountains['height'] + 50, # ยกป้ายลอยเหนือยอดนิดนึง
    mode='markers+text',
    marker=dict( #ตั้งค่ารายละเอียดของจุด
        size=5,
        color='red',
        symbol='diamond'
    ),
    text=real_mountains['name'] + "<br>" + real_mountains['height'].astype(str) + " ม.",
    textposition="top center",
    textfont=dict(
        size=12,
        color="black",
        family="Arial" # ตัวหนาอ่านง่าย
    ),
    name='ยอดเขา'
))

# 4. ตกแต่งกราฟ
fig.update_layout(
    title='จำลองโครงสร้างความสูงภูเขาไทย 3D (Surface Plot)',
    scene=dict(
        xaxis_title='ลองจิจูด',
        yaxis_title='ละติจูด',
        zaxis_title='ความสูง (เมตร)',
        aspectmode='manual',
        # ปรับสัดส่วนให้ Z สูงเด่นขึ้นมา (ยืดแกน Z)
        aspectratio=dict(x=1, y=1.5, z=0.4),
        camera=dict(
            eye=dict(x=1.5, y=-1.5, z=0.5) # มุมกล้องเริ่มต้น
        )
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    height=700
)

fig.show()